# Jupyter Notebook to estimate πN and πS

#### To estimate the diversity at synonymous (πS) and non-synonymous (πN) sites, the script dNdSpiNpiS (version 3, available at https://kimura.univ-montp2.fr/PopPhyl/index.php?section=tools) was used

#### The script requires a fasta file with the two diploid coding sequences of each individual containing SNPs

#### With header of sequences as: 

#### >CDS_1|Pearl_millet|Indiv1|Allele_1
#### >CDS_1|Pearl_millet|Indiv1|Allele_2

## 1. Obtain chromosome of each individual with the alternative alleles in accordance with a VCF file

## 2. Extract CDS from the ALT chromosome using a GFF file

In [ ]:
from Bio import SeqIO

file_ALT = str(individual)+"_ALL_CHR_FULL_IUPAC.fasta"

new_file_ALT = str(individual)+"_ALL_CHR_FULL_IUPAC_HEADER.fasta"

with open(new_file_ALT, "w") as new_fasta:
    for record in SeqIO.parse(file_ALT, "fasta"):
        header=str(">chr"+str(record.id)+"\n")
        new_fasta.write(header)
        seq=str(record.seq)
        new_fasta.write(seq+"\n")

dico_chr_ALT={}

for record in SeqIO.parse(new_file_ALT, "fasta"):
    if str(record.id) not in dico_chr_ALT:
        dico_chr_ALT[str(record.id)]=record.seq

fasta_file_REVERSE=str(individual)+"_ALL_CHR_FULL_IUPAC_CDS_reverse.fasta"
fasta_file_FORWARD=str(individual)+"_ALL_CHR_FULL_IUPAC_CDS_forward.fasta"

GFF_CDS="pearl_millet_23DB_ONT_assembly_annotation_transfer_POLISH_chr_scaff_GT_CDS_with_SNPS.gff"

count=1
count_reverse=0
list_CDS=[]
with open(fasta_file_REVERSE, "w") as fasta_to_generate_rev:
    with open(fasta_file_FORWARD, "w") as fasta_to_generate_for:
        with open(GFF_CDS, 'r') as gff:
            for line in gff:
                e=line.split("\t")
                chromosome=e[0]
                forward_reverse=e[6]
                if str(chromosome) in dico_chr_ALT: 
                    start=e[3] 
                    end=e[4] 
                    phase=e[7]
                    sequence=dico_chr_ALT[str(chromosome)][int(start)-1:int(end)]  
                    if str(forward_reverse) == "-":    
                        if str(phase) == "0" : 
                            sequence=sequence.reverse_complement()
                            fasta_to_generate_rev.write(">CDS"+str(count)+"_"+str(chromosome)+"_"+str(start)+"_"+str(end)+"\n"+str(sequence)+"\n")
                        elif str(phase) == "1" :
                            sequence=sequence.reverse_complement()
                            sequence=sequence[1:]
                            fasta_to_generate_rev.write(">CDS"+str(count)+"_"+str(chromosome)+"_"+str(start)+"_"+str(end)+"\n"+str(sequence)+"\n")
                        elif str(phase) == "2" : 
                            sequence=sequence.reverse_complement()
                            sequence=sequence[2:]
                            fasta_to_generate_rev.write(">CDS"+str(count)+"_"+str(chromosome)+"_"+str(start)+"_"+str(end)+"\n"+str(sequence)+"\n")
                                                          
                    if str(forward_reverse) == "+":   
                        if str(phase) == "0" : 
                            fasta_to_generate_for.write(">CDS"+str(count)+"_"+str(chromosome)+"_"+str(start)+"_"+str(end)+"\n"+str(sequence)+"\n")
                        elif str(phase) == "1" : 
                            sequence=sequence[1:]
                            fasta_to_generate_for.write(">CDS"+str(count)+"_"+str(chromosome)+"_"+str(start)+"_"+str(end)+"\n"+str(sequence)+"\n")
                        elif str(phase) == "2" : 
                            sequence=sequence[2:]                                
                            fasta_to_generate_for.write(">CDS"+str(count)+"_"+str(chromosome)+"_"+str(start)+"_"+str(end)+"\n"+str(sequence)+"\n")
                                                                         
                    count+=1

## 3. Obtain diploid sequences for each individual and each CDS

In [ ]:
# Forward CDS:

file_to_duplicate_seq_for=str(individual)+"_ALL_CHR_FULL_IUPAC_CDS_forward.fasta"
file_duplicated_forward=str(individual)+"_ALL_CHR_FULL_IUPAC_CDS_forward_DIPLOID.fasta"

no_change=['A','T','G','C']

count=1
with open(file_duplicated_forward, "w") as fR:
    for record in SeqIO.parse(str(file_to_duplicate_seq_for), "fasta"):
        sequence_1=""
        sequence_2=""
        id_seq=str(record.id) #>CDS85_chr1_3621451_3621710
        id_seq_e=id_seq.split("_")
        header=str(id_seq_e[1])+"_"+str(id_seq_e[2])+"_"+str(id_seq_e[3])
        SEQ=record.seq
        for car in SEQ:
            if str(car) == ".":
                sequence_1=str(sequence_1)+"N"
                sequence_2=str(sequence_2)+"N"
            if str(car) in no_change:
                sequence_1=str(sequence_1)+str(car)
                sequence_2=str(sequence_2)+str(car)
            if str(car) == "W":
                sequence_1=str(sequence_1)+"A"
                sequence_2=str(sequence_2)+"T"
            if str(car) == "S":
                sequence_1=str(sequence_1)+"C"
                sequence_2=str(sequence_2)+"G"
            if str(car) == "M":
                sequence_1=str(sequence_1)+"A"
                sequence_2=str(sequence_2)+"C"
            if str(car) == "K":
                sequence_1=str(sequence_1)+"G"
                sequence_2=str(sequence_2)+"T"
            if str(car) == "R":
                sequence_1=str(sequence_1)+"A"
                sequence_2=str(sequence_2)+"G"
            if str(car) == "Y":
                sequence_1=str(sequence_1)+"C"
                sequence_2=str(sequence_2)+"T"
        if  len(sequence_1) == len(sequence_2) and len(sequence_1) == len(SEQ): #check that sequences have the same length, as expected
            #>CDS_1|Pearl_millet|So-21-28371-02|Allele_1
            header_1=">CDS_"+str(count)+"_"+str(header)+"|Pearl_millet|"+str(individual)+"|Allele_1"
            fR.write(str(header_1)+"\n")
            fR.write(str(sequence_1)+"\n")
            header_2=">CDS_"+str(count)+"_"+str(header)+"|Pearl_millet|"+str(individual)+"|Allele_2"
            fR.write(str(header_2)+"\n")
            fR.write((sequence_2)+"\n")
        else:
            print(header)
        count+=1

# Reverse CDS:

file_to_duplicate_seq_rev=str(individual)+"_ALL_CHR_FULL_IUPAC_CDS_MANUAL_reverse.fasta"
file_duplicated_reverse=str(individual)+"_ALL_CHR_FULL_IUPAC_CDS_MANUAL_reverse_DIPLOID.fasta"

count=1
with open(file_duplicated_reverse, "w") as fR:
    for record in SeqIO.parse(str(file_to_duplicate_seq_rev), "fasta"):
        sequence_1=""
        sequence_2=""
        id_seq=str(record.id) #>CDS85_chr1_3621451_3621710
        id_seq_e=id_seq.split("_")
        header=str(id_seq_e[1])+"_"+str(id_seq_e[2])+"_"+str(id_seq_e[3])
        SEQ=record.seq
        for car in SEQ:
            if str(car) == ".":
                sequence_1=str(sequence_1)+"N"
                sequence_2=str(sequence_2)+"N"
            if str(car) in no_change:
                sequence_1=str(sequence_1)+str(car)
                sequence_2=str(sequence_2)+str(car)
            if str(car) == "W":
                sequence_1=str(sequence_1)+"A"
                sequence_2=str(sequence_2)+"T"
            if str(car) == "S":
                sequence_1=str(sequence_1)+"C"
                sequence_2=str(sequence_2)+"G"
            if str(car) == "M":
                sequence_1=str(sequence_1)+"A"
                sequence_2=str(sequence_2)+"C"
            if str(car) == "K":
                sequence_1=str(sequence_1)+"G"
                sequence_2=str(sequence_2)+"T"
            if str(car) == "R":
                sequence_1=str(sequence_1)+"A"
                sequence_2=str(sequence_2)+"G"
            if str(car) == "Y":
                sequence_1=str(sequence_1)+"C"
                sequence_2=str(sequence_2)+"T"
        if  len(sequence_1) == len(sequence_2) and len(sequence_1) == len(SEQ): #check that sequences have the same length, as expected
            #>CDS_1|Pearl_millet|So-21-28371-02|Allele_1
            header_1=">CDS_"+str(count)+"_"+str(header)+"|Pearl_millet|"+str(individual)+"|Allele_1"
            fR.write(str(header_1)+"\n")
            fR.write(str(sequence_1)+"\n")
            header_2=">CDS_"+str(count)+"_"+str(header)+"|Pearl_millet|"+str(individual)+"|Allele_2"
            fR.write(str(header_2)+"\n")
            fR.write((sequence_2)+"\n")
        else:
            print(header)
        count+=1

## 4. Concatenation of all the CDS for all individuals per chromosome

#### [INDIV]_ALL_CHR_FULL_IUPAC_CDS_reverse_DIPLOID.fasta
#### header: >CDS_100_chr1_40274075_40276121|Pearl_millet|So-21-28371-04|Allele_1

#### => Concatenation of all the CDS of all individuals together, per chromosome 

#### => files: CHR*_forward_reverse.fasta

In [ ]:
# grep ">" So-8-7539-04_ALL_CHR_FULL_IUPAC_CDS_forward_DIPLOID.fasta | wc -l
# 6736
# grep ">" So-8-7539-04_ALL_CHR_FULL_IUPAC_CDS_reverse_DIPLOID.fasta | wc -l
# 6950

In [ ]:
list_of_individuals=["So-21-28371-02","So-21-28371-03","So-21-28371-04","So-21-29822-01","So-21-29822-02","So-21-30529-01","So-21-30529-03","Sa-8-12129-01","Sa-8-12129-02","Sa-8-12129-03","Sa-8-12129-04","So-10-20006-01","So-10-20006-02","So-10-20110-02","So-10-20110-03","So-10-20117-01","So-10-20117-03","So-10-20279-01","So-10-20279-02","So-10-20279-03","So-10-20302-01","So-10-20302-02","So-10-20302-03","So-10-20327-01","So-10-20327-02","So-10-20327-03","So-10-20333-01","So-10-20333-02","So-10-20333-04","So-10-20333-05","So-10-20347-01","So-10-20347-02","So-10-20347-03","So-10-20363-01","So-10-20363-02","So-10-20363-03","So-10-56965-01","So-10-56965-02","So-10-56965-03","Sa-8-11559-01","So-21-1061629-02","So-21-28371-01","Sa-21-28423-03","Sa-21-82410-01","So-8-6858-03","Sa-21-30147-01","Sa-6-4253-02","So-6-51263-01","So-21-30507-01","So-21-28423-01","Sa-21-82410-02","So-21-28425-01","So-8-7291-01","Sa-21-30147-02","So-6-4253-01","So-6-51263-02","So-21-30507-02","So-21-28423-02","Sa-21-82410-03","So-21-28425-02","Sa-21-30147-03","So-6-4253-02","Sa-6-3385-01","So-21-28423-03","Sa-21-82410-04","So-21-28425-03","So-8-7539-01","Sa-21-30147-04","So-6-3189-01","Sa-21-30507-01","Sa-21-30218-01","So-8-6685-01","So-8-7539-02","Sa-8-7085-01","So-6-3189-02","So-6-3385-02","Sa-21-30507-02","Sa-21-30218-02","So-8-6685-02","So-8-7539-03","Sa-8-7085-02","Sa-6-4253-01","Sa-6-51263-02","Sa-21-30529-03","So-8-6858-02","Sa-8-11559-03","So-6-3189-03","So-6-4452-01","Sa-24-60424-03","Sa-21-30218-03","So-8-6685-03","So-8-7539-04","Sa-8-7085-03","Sa-6-3189-01","So-21-30522-01","So-21-30218-01","So-8-7024-01","So-8-11559-01","Sa-8-7085-04","Sa-6-3189-02","Sa-21-30515-01","So-21-30522-02","So-21-28449-01","So-21-30218-02","So-8-7024-02","So-8-11559-02","Sa-8-7085-05","Sa-6-3189-03","Sa-21-30515-02","So-21-30522-03","So-21-28449-02","So-21-30218-03","So-8-7024-03","So-8-11559-03","Sa-8-7085-06","So-21-1061629-01","Sa-6-3189-04","Sa-21-30529-02","Sa-21-28423-02","So-21-28449-04","Sa-21-28425-02","So-21-30529-02","So-10-20006-03","So-10-20333-03","So-21-1061629-03","Sa-24-60424-01"]

In [ ]:
from Bio import SeqIO

# Getting of all the individuals for each CDS:

nb_CDS_forward=6736/2
nb_CDS_reverse=6950/2

for i in range(1,int(nb_CDS_forward+1)):
    with open("results/CDS/CDS"+str(i)+"_forward.fasta", "w") as all_cds_forward_chr:
        for indiv in list_of_individuals:
            for record in SeqIO.parse("results/"+str(indiv)+"_ALL_CHR_FULL_IUPAC_CDS_forward_DIPLOID.fasta", "fasta"):
                header=record.id #>CDS_1_chr1_421212_421553|Pearl_millet|So-21-1061629-03|Allele_1
                e=header.split("_")
                cds_id=e[1]
                if str(cds_id) == str(i):
                    seq=str(record.seq)
                    all_cds_forward_chr.write(">"+header+"\n")
                    all_cds_forward_chr.write(seq+"\n")
                    
for i in range(1,int(nb_CDS_reverse+1)):
    with open("results/CDS/CDS"+str(i)+"_reverse.fasta", "w") as all_cds_forward_chr:
        for indiv in list_of_individuals:
            for record in SeqIO.parse("results/"+str(indiv)+"_ALL_CHR_FULL_IUPAC_CDS_reverse_DIPLOID.fasta", "fasta"):
                header=record.id #>CDS_1_chr1_421212_421553|Pearl_millet|So-21-1061629-03|Allele_1
                e=header.split("_")
                cds_id=e[1]
                if str(cds_id) == str(i):
                    seq=str(record.seq)
                    all_cds_forward_chr.write(">"+header+"\n")
                    all_cds_forward_chr.write(seq+"\n")

In [ ]:
# cat CDS*_reverse.fasta > all_CDS_reverse.fasta
# cat CDS*_forward.fasta > all_CDS_forward.fasta

In [ ]:
from Bio import SeqIO

with open("results/CDS/chr1_reverse.fasta", "w") as all_cds_reverse_chr:
    for record in SeqIO.parse("results/CDS/all_CDS_reverse.fasta", "fasta"):
        header=record.id #CDS_1000_chr2_244263702_244263978|Pearl_millet|So-21-28371-02|Allele_1
        e=header.split("_")
        chromosome=e[2]
        if str(chromosome) == "chr1":
            seq=str(record.seq)
            all_cds_reverse_chr.write(">"+header+"\n")
            all_cds_reverse_chr.write(seq+"\n") 
            
with open("results/CDS/chr1_forward.fasta", "w") as all_cds_forward_chr:
    for record in SeqIO.parse("results/CDS/all_CDS_forward.fasta", "fasta"):
        header=record.id #CDS_1000_chr2_244263702_244263978|Pearl_millet|So-21-28371-02|Allele_1
        e=header.split("_")
        chromosome=e[2]
        if str(chromosome) == "chr1":
            seq=str(record.seq)
            all_cds_forward_chr.write(">"+header+"\n")
            all_cds_forward_chr.write(seq+"\n") 

In [ ]:
# cat chr1_forward.fasta chr1_reverse.fasta > CHR1_forward_reverse.fasta

## 5. Remove sequences with N

In [ ]:
file="CHR1_forward_reverse.fasta"
new_file="CHR1_forward_reverse_no_indiv_with_N.fasta"

In [ ]:
from Bio import SeqIO

with open(new_file, "w") as new:
    for record in SeqIO.parse(file, "fasta"):
        sequence=record.seq
        header=">"+str(record.id)
        if "N" not in sequence:
            new.write(header+"\n")
            new.write(str(sequence)+"\n")

## 6. Launch the script dNdSpiNpiS_1.0 per chromosome

In [ ]:
# ./dNdSpiNpiS_1.0 -alignment_file=CHR*_forward_reverse_no_indiv_with_N.fasta -ingroup=Pearl_millet
# CHR*_forward_reverse_no_indiv_with_N.out
# CHR*_forward_reverse_no_indiv_with_N.sum

# cat CHR*_forward_reverse_no_indiv_with_N.out > ALL_CHR_forward_reverse_no_indiv_with_N.out

## 7. Get the corrector factor = the percentage of bases called per CDS

### Get the positions of the variant and unvariant sites

In [ ]:
list_called_position=[]

with open("chr1_all_sites_variant_unvariant_header.vcf", 'r') as f:
    for line in f:
        e=line.split("\t")
        if str(e[0][0]) != "#":
            position=e[1]
            list_called_position.append(position)

In [ ]:
list_CDS=[]
with open("CHR*_forward_reverse_no_indiv_with_N.out", 'r') as f:
    for l in f:
        e=l.split("\t")
        contig=e[0]
        if str(contig) != "Contig_name":
            if str(contig) not in list_CDS:
                list_CDS.append(contig)

In [ ]:
dico_CDS_callable={}
for CDS in list_CDS: # CDS_1000_chr2_244263702_244263978
    dico_CDS_callable[CDS]={}
    e=CDS.split("_")
    start=e[3] # start of a CDS
    end=e[4] # end of a CDS
    length_cds=int(end)-int(start)
    dico_CDS_callable[CDS]["length"]=length_cds
    nb_callable_sites=0
    for position in list_called_position: # if a called position is within a CDS:
        if int(position) > int(start) and int(position) < int(end): 
            nb_callable_sites+=1
        dico_CDS_callable[CDS]["nb_callable_sites"]=nb_callable_sites
        percentage_called=float(nb_callable_sites/length_cds)
        dico_CDS_callable[CDS]["precentage_called"]=percentage_called

## 8.1 Mean diversity all genome and bootstrap

In [ ]:
sum_pi_N=0.0
sum_pi_S=0.0

dico_for_bootstrap={}

c=0
with open("./final_all_data.txt","w") as fR:
    fR.write("Contig\tp_called\tcorr_factor\tpiN\tpiS\tpiN_corr\tpiS_corr\n")
    with open("/ALL_CHR_forward_reverse_no_indiv_with_N.out", "r") as f:
        for line in f:
            e=line.split("\t")
            contig=e[0]
            if str(e[0])!="Contig_name":
                p_called=float(dico_CDS_callable[contig]["precentage_called"])
                if float(p_called) > 0.0:
                    dico_for_bootstrap[c]={}
                    f_corr=float(1.0/float(p_called))
                    contig=e[0]
                    piN=e[26]
                    new_piN=float(piN)*float(f_corr)
                    sum_pi_N+=float(new_piN)
                    piS=e[27]
                    new_piS=float(piS)*float(f_corr)
                    sum_pi_S+=float(new_piS)
                    sum_factor+=float(p_called)
                    dico_for_bootstrap[c]["contig"]=contig
                    dico_for_bootstrap[c]["new_piN"]=new_piN
                    dico_for_bootstrap[c]["new_piS"]=new_piS
                    fR.write(str(contig)+"\t"+str(p_called)+"\t"+str(f_corr)+"\t"+str(piN)+"\t"+str(piS)+"\t"+str(new_piN)+"\t"+str(new_piS)+"\n")
                    c+=1
                        
mean_piN=float(sum_pi_N)/float(c)
mean_piS=float(sum_pi_S)/float(c)
print(mean_piN)
print(mean_piS)
ratio=float(mean_piN)/float(mean_piS)
print(ratio)

0.003723955763111119
0.011901595381950526
0.3128955105261534
6453


### Bootstrap:

In [ ]:
import random

list_nb=[]
list_mean_piN=[]
list_mean_piS=[]
list_ratio=[]

nb_tests=1000

for z in range(0,nb_tests):
    sum_piN=0.0
    sum_piS=0.0
    sum_p_called=0.0
    c=0
    for i in range(0,6452):
        nb=random.randint(0,6452)
        piN=dico_for_bootstrap[nb]["new_piN"]        
        piS=dico_for_bootstrap[nb]["new_piS"]
        sum_piS+=float(piS)
        sum_piN+=float(piN)
        c+=1
    mean_diversity_N=float(sum_piN)/float(c)
    mean_diversity_S=float(sum_piS)/float(c)
    ratio_boot=float(mean_diversity_N)/float(mean_diversity_S)
    list_mean_piN.append(mean_diversity_N)
    list_mean_piS.append(mean_diversity_S)
    list_ratio.append(ratio_boot)
list_mean_piN.sort()
list_mean_piS.sort()
list_ratio.sort()

In [ ]:
print(list_mean_piN[24])
print(list_mean_piN[974])

In [ ]:
print(list_mean_piS[24])
print(list_mean_piS[974])

In [ ]:
print(list_ratio[24])
print(list_ratio[974])

## 8.2 Mean diversity per region

In [149]:
# chr1
# 99793192
# 161758608
# chr2
# 131088113
# 186649394
# chr3
# 101238580
# 188834466
# chr4
# 118491302
# 160157576
# chr6_1
# 132717319
# 201482742
# chr6_2
# 224133144
# 229409049
# chr7
# 128666580
# 177591959

sum_pi_N=0.0
sum_pi_S=0.0
sum_factor=0.0

start_candidate_region=101238580
end_candidate_region=188834466

c=0
with open("./out_put_scripts/ALL_CHR_forward_reverse_no_indiv_with_N.out", "r") as f:
    for line in f:
        e=line.split("\t")
        if str(e[0])!="Contig_name":
            contig=e[0]
            info=contig.split("_")
            chromosome=info[2]
            start=info[3]
            end=info[4]
            p_called=float(dico_factor_corr[contig]["factor"])
            if str(chromosome) == "chr3":
                if int(start) > int(start_candidate_region) and int(start) < int (end_candidate_region): 
                    if float(p_called) > 0.0:
                        c+=1
                        f_corr=float(1.0/float(p_called))
                        contig=e[0]
                        piN=e[26]
                        new_piN=float(piN)*float(f_corr)
                        sum_pi_N+=float(new_piN)
                        piS=e[27]
                        new_piS=float(piS)*float(f_corr)
                        sum_pi_S+=float(new_piS)
                        sum_factor+=float(p_called)
                        #print(new_piN,new_piS)
                        
print(c)
mean_piN=float(sum_pi_N)/float(c)
mean_piS=float(sum_pi_S)/float(c)
print(mean_piN)
print(mean_piS)
ratio=float(mean_piN)/float(mean_piS)
print(ratio)

75
0.004851244558997575
0.01010781069902459
0.4799500805318528


## Boostrap analysis and resampling procedure for the regions:

In [ ]:
# chr1
# 99793192
# 161758608
# chr2
# 131088113
# 186649394
# chr3
# 101238580
# 188834466
# chr4
# 118491302
# 160157576
# chr6_1
# 132717319
# 201482742
# chr6_2
# 224133144
# 229409049
# chr7
# 128666580
# 177591959

In [126]:
start_candidate_region=128666580
end_candidate_region=177591959
chromosome_teste="chr7"

dico_for_bootstrap={}

c=0
with open("./out_put_scripts/final/final_all_data.txt", "r") as f:
    for line in f:
        e=line.split("\t")
        contig=e[0]
        if str(e[0])!="Contig":
            info=contig.split("_")
            chromosome=info[2]
            start=info[3]
            end=info[4]
            if str(chromosome) == str(chromosome_teste):
                if int(start) > int(start_candidate_region) and int(start) < int (end_candidate_region): 
                    dico_for_bootstrap[c]={}
                    new_piN=e[5] #new_piN /!\
                    new_piS=e[6] #new_piN /!\
                    f_corr=e[2]
                    p_called=e[1]
                    dico_for_bootstrap[c]["contig"]=contig
                    dico_for_bootstrap[c]["new_piN"]=new_piN
                    dico_for_bootstrap[c]["new_piS"]=new_piS
                    dico_for_bootstrap[c]["f_corr"]=f_corr
                    dico_for_bootstrap[c]["p_called"]=p_called
                    c+=1


In [127]:
print(len(dico_for_bootstrap))
#print(dico_for_bootstrap)
nb_cds_region=int(len(dico_for_bootstrap))

28


In [128]:
print(c)

28


In [129]:
import random

list_nb=[]
list_mean_piN=[]
list_mean_piS=[]
list_ratio=[]

for z in range(0,1000):
    sum_piN=0.0
    sum_piS=0.0
    sum_p_called=0.0
    for i in range(0,int(nb_cds_region)-1):
        nb=random.randint(0,int(nb_cds_region)-1)
        piN=dico_for_bootstrap[nb]["new_piN"]        
        piS=dico_for_bootstrap[nb]["new_piS"]
        p_called=dico_for_bootstrap[nb]["p_called"]
        sum_piS+=float(piS)
        sum_piN+=float(piN)
        sum_p_called+=float(p_called)
    mean_diversity_N=float(sum_piN)/float(c)
    mean_diversity_S=float(sum_piS)/float(c)
    list_mean_piN.append(mean_diversity_N)
    list_mean_piS.append(mean_diversity_S)
    ratio_boot=float(mean_diversity_N)/float(mean_diversity_S)
    list_ratio.append(ratio_boot)

list_mean_piN.sort()
list_mean_piS.sort()
list_ratio.sort()

In [130]:
print(len(list_mean_piN))
print(len(list_mean_piS))
print(len(list_ratio))

1000
1000
1000


In [131]:
print(list_ratio[24])
print(list_ratio[974])

0.2587849331878365
1.756649087963642


In [132]:
print(list_mean_piN[24])
print(list_mean_piN[974])

0.003436509115860871
0.008914685120636309


In [133]:
print(list_mean_piS[24])
print(list_mean_piS[974])

0.0034600523308874445
0.017457396664200243


## Resampling procedure

In [96]:
# chr1
# 99793192
# 161758608
# chr2
# 131088113
# 186649394
# chr3
# 101238580
# 188834466
# chr4
# 118491302
# 160157576
# chr6_1
# 132717319
# 201482742
# chr6_2
# 224133144
# 229409049
# chr7
# 128666580
# 177591959

In [134]:
dico_for_resampling={}

c=0

sum_piN=0
sum_piS=0

c=0

list_excluded_cds=[]

i=0
with open("./out_put_scripts/final/final_all_data.txt", "r") as f:
    for line in f:
        e=line.split("\t")
        contig=e[0]
        if str(e[0])!="Contig":
            info=contig.split("_")
            chromosome=info[2]
            start=info[3]
            end=info[4]
            if str(chromosome) == str(chromosome_teste):
                if int(start) > int(start_candidate_region) and int(start) < int (end_candidate_region): 
                    c+=1
                    new_piN=e[5] #new_piN /!\
                    new_piS=e[6] #new_piN /!\
                    sum_piN+=float(new_piN)
                    sum_piS+=float(new_piS)
                    list_excluded_cds.append(i)
            i+=1
mean_piN_region=float(sum_piN)/float(c)
mean_piS_region=float(sum_piS)/float(c)
mean
nb_cds=c
print("piN:",mean_piN_region)
print("piS:",mean_piS_region)
print("nbre de CDS:", nb_cds)

piN: 0.006026999416800385
piS: 0.009938425985307253
nbre de CDS: 28


In [135]:
print(list_excluded_cds)

[5875, 5876, 5877, 5878, 5879, 5880, 5881, 5882, 5883, 5884, 5885, 5886, 5887, 5888, 5889, 5890, 5891, 6295, 6296, 6297, 6298, 6299, 6300, 6301, 6302, 6303, 6304, 6305]


In [136]:
dico_resampling={}
i=0
c=0
with open("./out_put_scripts/final/final_all_data.txt", "r") as f:
    for line in f:
        e=line.split("\t")
        contig=e[0]
        if str(e[0])!="Contig":
            if int(i) not in list_excluded_cds:
                dico_resampling[c]={}
                new_piN=e[5] #new_piN /!\
                new_piS=e[6] #new_piN /!\
                dico_resampling[c]["contig"]=contig
                dico_resampling[c]["piN"]=float(new_piN)
                dico_resampling[c]["piS"]=float(new_piS)
                c+=1
            i+=1
print(len(dico_resampling))
print(dico_resampling[0])

6425
{'contig': 'CDS_100_chr1_40274075_40276121', 'piN': 0.0033369593207547166, 'piS': 0.011938673207547169}


In [139]:
import random

list_mean_piN=[]
list_mean_piS=[]
list_mean_ratio=[]

for z in range(0,1000):
    sum_piN=0.0
    sum_piS=0.0
    for i in range(0,int(nb_cds)-1):
        nb=random.randint(0,len(dico_resampling)-1)
        piN=dico_resampling[nb]["piN"]
        piS=dico_resampling[nb]["piS"]
        sum_piN+=float(piN)
        sum_piS+=float(piS)
    mean_piN=float(sum_piN)/float(nb_cds)
    mean_piS=float(sum_piS)/float(nb_cds)
    ratio=float(mean_piN)/float(mean_piS)
    list_mean_ratio.append(ratio)
    list_mean_piN.append(float(mean_piN))
    list_mean_piS.append(float(mean_piS))

list_mean_piN_sort=sorted(list_mean_piN, key = lambda x:float(x))
list_mean_piS_sort=sorted(list_mean_piS, key = lambda x:float(x))
list_mean_ratio_sort=sorted(list_mean_ratio, key = lambda x:float(x))
print(len(list_mean_piN_sort))
print(len(list_mean_piS_sort))
print(len(list_mean_ratio_sort))

count_rank=0
print("\npiN:")

print("min:",min(list_mean_piN_sort))
print("mean region:",mean_piN_region)
print("max:",max(list_mean_piN_sort),"\n")

for mean in list_mean_piN_sort:
    if float(mean_piN_region) >= float(mean):
        count_rank+=1
print(count_rank)

count_rank=0
for mean in list_mean_piS_sort:
    if float(mean_piS_region) >= float(mean):
        count_rank+=1
print("\npiS:")
print("min:",min(list_mean_piS_sort))
print("mean region:",mean_piS_region)
print("max:",max(list_mean_piS_sort),"\n")
print(count_rank)

count_rank=0
mean_ratio=0.6064339992782123
for mean in list_mean_ratio_sort:
    if float(mean_ratio) >= float(mean):
        count_rank+=1
print("\nRATIO:")
print("min:",min(list_mean_ratio_sort))
print("mean region:",mean_ratio)
print("max:",max(list_mean_ratio_sort),"\n")
print(count_rank)

1000
1000
1000

piN:
min: 0.0012655323676580617
mean region: 0.006026999416800385
max: 0.007654614460578837 

978

piS:
min: 0.00398924739542613
mean region: 0.009938425985307253
max: 0.027548076364746342 

347

RATIO:
min: 0.0681070330403458
mean region: 0.6064339992782123
max: 1.409681816960023 

935
